In [2]:
import torch
import numpy as np

In [3]:
atomic_number = 21
batch_size = 60

In [9]:
position_batch_center = torch.randn(batch_size, atomic_number, 3)
atomic_adjacency_matrix = torch.zeros(batch_size, atomic_number, atomic_number)
connections = [(0, 1), (1, 2), (2, 3), (4, 5), 
               (5, 6), (18, 20)
               ]  # 假设这些原子对之间存在连接

# 将连接关系转换为张量
# 注意：这里我们使用了torch.tensor()来转换列表，并确保数据类型与索引匹配（通常是long）
rows = torch.tensor([i for i, j in connections], dtype=torch.long)
cols = torch.tensor([j for i, j in connections], dtype=torch.long)

# 使用高级索引同时更新邻接矩阵的对称位置
# 这里我们遍历每个批次进行更新，但更新操作本身利用了广播机制
for b in range(batch_size):
    atomic_adjacency_matrix[b, rows, cols] = 1
    atomic_adjacency_matrix[b, cols, rows] = 1  # 确保对称性

# 打印结果以验证
print("Modified Adjacency Matrix:\n", atomic_adjacency_matrix)

Modified Adjacency Matrix:
 tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.]],

        [[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.]],

        [[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.]],

        ...,

        [[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0.,

In [18]:
def coordinates2localframe_type1(position_batch_center, atomic_adjacency_matrix, norm_diff=True):
    batch_size, num_atoms, _ = position_batch_center.shape

    # 扩展维度以进行广播
    coords_row = position_batch_center.unsqueeze(2).expand(-1, -1, num_atoms, -1)
    coords_col = position_batch_center.unsqueeze(1).expand(-1, num_atoms, -1, -1)
    # 计算坐标差异
    coord_diff = coords_row - coords_col
    # batch_size x num_atoms x num_atoms x 3 
    # (1, 1, 2, :)代表的第二个批次里面第二个原子和第三个原子的位置差
    # # 计算径向距离
    ## TODO: 问题出在这里， 进行cross的计算会导致两个数值接近0，从而导致nan的出现。
    ## TODO: 选取dummy data 来做测试来保证矩阵形状差不多。
    radial = torch.sum(coord_diff ** 2, dim=-1, keepdim=True)
    # coord_cross 的矩阵也是 batch_size x num_atoms x num_atoms x 3 
    # (1, 1, 2, :) 表示第二个批次里面的第二个原子的向量和和第三个原子的向量的cross
    coord_cross = torch.randn(batch_size, num_atoms, num_atoms, 3)
    for i in range(num_atoms):
        for j in range(num_atoms):
            pos_i = position_batch_center[:, i, :]
            pos_j = position_batch_center[:, j, :]
            coord_cross[:, i, j, :] = torch.cross(pos_i, pos_j)

    # coord_cross = torch.cross(coord_diff, coords_row, dim=-1)  # 这里应保持coord_diff作为叉乘的输入
    # 规范化向量
    if norm_diff:
        norm = torch.sqrt(radial) + 1e-5
        coord_diff = coord_diff / norm
        cross_norm = torch.sqrt(torch.sum(coord_cross ** 2, dim=-1, keepdim=True)) + 1e-5
        coord_cross = coord_cross / cross_norm
    # 计算第三个轴
    coord_vertical = torch.cross(coord_diff, coord_cross, dim=-1)
    # 使用邻接矩阵筛选出相邻的原子对
    mask = atomic_adjacency_matrix.unsqueeze(-1)  # 扩展维度以匹配
    coord_diff = coord_diff * mask
    coord_cross = coord_cross * mask
    coord_vertical = coord_vertical * mask
    # 三者都是batch_size x num_atoms x num_atoms x 3
    return coord_diff, coord_cross, coord_vertical
coord_diff, coord_cross, coord_vertical = coordinates2localframe_type1(position_batch_center, atomic_adjacency_matrix, norm_diff=True)

In [19]:
def coordinates2localframe_type2(position_batch_center, atomic_adjacency_matrix, norm_diff=True):
    batch_size, num_atoms, _ = position_batch_center.shape

    # 扩展维度以进行广播
    coords_row = position_batch_center.unsqueeze(2).expand(-1, -1, num_atoms, -1)
    coords_col = position_batch_center.unsqueeze(1).expand(-1, num_atoms, -1, -1)
    # 计算坐标差异
    coord_diff = coords_row - coords_col
    # batch_size x num_atoms x num_atoms x 3 
    radial = torch.sum(coord_diff ** 2, dim=-1, keepdim=True)
    coord_cross = torch.cross(coords_row, coords_col, dim=-1)
    # 规范化向量
    if norm_diff:
        norm = torch.sqrt(radial) + 1e-5
        coord_diff = coord_diff / norm
        cross_norm = torch.sqrt(torch.sum(coord_cross ** 2, dim=-1, keepdim=True)) + 1e-5
        coord_cross = coord_cross / cross_norm
    # 计算第三个轴
    coord_vertical = torch.cross(coord_diff, coord_cross, dim=-1)
    # 使用邻接矩阵筛选出相邻的原子对
    mask = atomic_adjacency_matrix.unsqueeze(-1)  # 扩展维度以匹配
    coord_diff = coord_diff * mask
    coord_cross = coord_cross * mask
    coord_vertical = coord_vertical * mask
    # 三者都是batch_size x num_atoms x num_atoms x 3
    return coord_diff, coord_cross, coord_vertical
_coord_diff, _coord_cross, _coord_vertical = coordinates2localframe_type2(position_batch_center, atomic_adjacency_matrix, norm_diff=True)

In [24]:
torch.sum(coord_cross-_coord_cross)

tensor(0.)

In [16]:
# Execute the function with dummy data
coord_diff, coord_cross, coord_vertical = coordinates2localframe(position_batch_center, atomic_adjacency_matrix, norm_diff=True)

# Create a dummy loss and compute gradients to check for gradient flow
loss = torch.sum(coord_diff) + torch.sum(coord_cross) + torch.sum(coord_vertical)
loss.backward()

# Check gradients exist for the input tensor
print("Gradient for position_batch_center:", position_batch_center.grad)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [1]:
import numpy as np

In [2]:
npz_path = r"C:\Users\23174\Desktop\GitHub Project\GitHubProjectBigData\GNN-Molecular-Project\GNN-LF-AND-ColfNet\dl_gnn\data\MD17\raw\md17_aspirin.npz"
data= np.load(npz_path)

In [4]:
for key in data.keys():
    print(key, data[key].shape)

E (211762, 1)
name ()
F (211762, 21, 3)
theory ()
R (211762, 21, 3)
z (21,)
type ()
md5 ()
